#  News Content Collect and Store
## by Aly Abdelrazek
 
 


##Importing libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pickle


##Global variables

In [2]:
news_page_url = "https://bbc.com/news"
absolute_link_prefix = "https://bbc.com"


##Scrape


In [3]:
# Obtains news stories links from news_page_url (https://bbc.com/news)
def news_page_to_news_urls(url, prefix):
    ''' Takes the URL to a news page and the prefix of the absolute link, and returns a list of news stories URLs.'''
    news_urls = []
    news_page_html = requests.get(url).text
    news_page_soup = BeautifulSoup(news_page_html, "html.parser")
    news_page_links = news_page_soup.findAll("a") #find all links
    
    for link in news_page_links:
        href = link.get("href")
        if href.startswith("/news") and href[-1].isdigit(): #news articles start with /news and ends with a digit
            news_url = prefix + href
            s = BeautifulSoup(requests.get(news_url).text , "html.parser")
            if s.find(class_= "css-16rg7hm-ContainerWithSidebarWrapper e1jl38b40") is not None: #this class is only available in news articles
              news_urls.append(news_url)
    return news_urls

news_urls = news_page_to_news_urls(news_page_url, absolute_link_prefix)

## Cleanse

In [ ]:
news_headings = []
news_texts =[]
news_times = []

for url in news_urls:
    print("Fetching {}".format(url))
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    news_heading = (soup.find('h1')).text
    news_text = [p.text for p in soup.find('article').find_all('p')]
    news_time = (soup.find('time'))['datetime']

    news_headings.append(news_heading)
    news_times.append(np.datetime64(news_time)) 
    news_texts.append(news_text)

In [45]:
df = pd.DataFrame(list(zip(news_times, news_headings, news_texts)), 
                  columns =['datetime', 'heading', 'text']) 